In [141]:

import torch
from transformers import LlamaForCausalLM, AutoModelForCausalLM,AutoTokenizer, LlamaTokenizer, LlamaTokenizerFast
import json
model_name_or_path = "/local1/zefan/output/NewGenDatav2_400_10definition/epoch_7"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if isinstance(tokenizer, LlamaTokenizer) or isinstance(tokenizer, LlamaTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
    })
    assert num_added_tokens in [0, 1], "LlamaTokenizer should only add one special token - the pad_token, or no tokens if pad token present."


cnt_error = 0
pos = 0
neg = 0
seq_len = []
with open("../data/generated_data/train_400_10definitions_v2.json", "r") as F:
    for line in F.readlines():
        dat = json.loads(line)
        sentence = dat['prompt'].split("SENTENCE:")[1].split("EVENT TYPE")[0].strip()
        trigger = dat['completion'].replace("Event trigger is ", "").replace(".", "").strip()
        if trigger != "<trigger>":
            pos +=1
            if not trigger in sentence:
                cnt_error += 1
            seq_len.append(len(tokenizer(dat["prompt"] + dat["completion"]).input_ids))   
            if seq_len[-1] > 240:
                print(dat["prompt"])
        else:
            neg +=1
        
print(max(seq_len))
print(cnt_error, pos, neg)

198
0 3908 39080


In [142]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path).to("cuda:0")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.71s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /local1/zefan/output/NewGenDatav2_400_10definition/epoch_7 and are newly initialized: ['model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn

In [147]:
# tokenizer(['SENTENCE: A senior administration official said the president had decided to be cautious . \n EVENT TYPE: divorce. \n DEFINITION: A DIVORCE Event occurs whenever two people are officially divorced under the legal definition of divorce. We do not include separations or church annulments. \n PARENT: life, SON: be born, marry, divorce, injure, die. \n So what is the trigger? Event trigger is']).input_ids
# tokenizer.decode(
#     [32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
#          32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
#          32003,   317,  3919,  1430,  4741, 29901,   319, 16336, 17517,  6221,
#           1497,   278,  6673,   750,  8459,   304,   367,   274,  1300,  2738,
#            869, 29871,    13,   382, 29963,  3919,   323,  6959, 29901, 25074,
#            346, 29889, 29871,    13,  5012, 29943,  1177, 22122, 29901,   319,
#            360,  5667,  1955,  4741,  6864, 10008, 10940,  1023,  2305,   526,
#          22444, 25074,  1133,  1090,   278, 11706,  5023,   310, 25074,   346,
#          29889,  1334,   437,   451,  3160,  2903,   800,   470,  6586,  2889,
#            352,  1860, 29889, 29871,    13,   349,  1718,  3919, 29901,  2834,
#          29892,   317,  1164, 29901,   367,  6345, 29892, 20479, 29892, 25074,
#            346, 29892, 10899,   545, 29892,   762, 29889, 29871,    13,  1105,
#            825,   338,   278,  7135, 29973,  6864,  7135,   338,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
# )
# tokenizer.decode([3919])

input_ids = [[32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003,     1,   317,  3919,  1430,
          4741, 29901, 16704,   423,   674,  9055,   278,  8225,   310,  4832,
          7602,   919, 29879,  3704,   385,  7560,   373,  4892,  1948,  1156,
          5320,   310,   963,  5929,  7943,   304,   278, 22569,  9245,   363,
           263,  1473,  9076,  1919,   410,  3471, 29560,  1497,   498,  1295,
          3250,   869, 29871,    13,   382, 29963,  3919,   323,  6959, 29901,
          6782,  6909, 29889, 29871,    13,  5012, 29943,  1177, 22122, 29901,
         10014,  2190, 20322,  1001, 29899, 29924, 12413, 29979, 28488,  2737,
           304,   278,  6820, 29892, 13442, 29892, 27942,   292, 29892,   470,
           301,  2548,  6909,   746,   372,   338,   451,   297,   278,  3030,
           310, 10596,  5832,  1554, 29889,   450, 24420,  6455,   526, 29901,
           313, 29896, 29897,  2305,  6820,  6909,   304, 25700,   313,   392,
          2805,  3078, 18806,  1821,   297,   736,   416,   322,   313, 29906,
         29897, 25700,   301,  2548,  6909,   304,  2305,   470,   916,  1638,
         29879, 29889, 29871,    13,   349,  1718,  3919, 29901, 10804, 29892,
           317,  1164, 29901,  6782, 27428, 29892,  6782,  6909, 29889, 29871,
            13,  1105,   825,   338,   278,  7135, 29973,  6864,  7135,   338]]
input_ids = torch.LongTensor(input_ids).to("cuda:0")
attention_mask = input_ids != 32003

print(sum([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]), torch.sum(attention_mask))

output = model.generate(
  input_ids = input_ids,
  attention_mask = attention_mask,
  max_new_tokens = 10,
  stopping_criteria = None)
print(output)



157 tensor(164, device='cuda:0')
tensor([[32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003, 32003,
         32003, 32003, 32003, 32003, 32003, 32003,     1,   317,  3919,  1430,
          4741, 29901, 16704,   423,   674,  9055,   278,  8225,   310,  4832,
          7602,   9

In [145]:
a = None
if a != []:
    print("AAA")

AAA


In [109]:
import json
data = json.load(open("data/generated_data_fix_v4.json", 'r'))

In [110]:
to_remove_parent_list = []
for parent in data:
    if len(data[parent]['events']) > 8:
        to_remove_parent_list.append(parent)
for parent in to_remove_parent_list:
    del data[parent]

In [114]:
# print(data.keys())
ontology_size = []
events = {}
total_events = []
to_remove = []
for parent in data:
    ontology_size.append(len(data[parent]['events']))
    for ev in data[parent]['events']:
        if ev in events:
            events[ev].append([parent, ontology_size[-1]])
        else:
            events[ev] = [[parent, ontology_size[-1]]]
    total_events += data[parent]['events']
    # if ontology_size[-1] > 7:
    #     print(parent, data[parent]['events'])
print(len(total_events), len(set(total_events)))
ontology_size = np.array(ontology_size)
print(len(ontology_size))
print(np.sum(ontology_size > 7))

for ev, parents in events.items():
    events[ev] = sorted(parents, key=lambda x: x[1], reverse=True)
remove_parent_num = 0
remove_child_num = 0
for ev, parents in events.items():
    if len(parents) > 1:
        for to_re in parents[1:]:
            to_remove_parent = to_re[0]
            to_remove_events = ev
            if len(data[to_remove_parent]['events']) <= 1: #only one event, remove parents
                del data[to_remove_parent]
                remove_parent_num += 1
            else:
                remove_child_num +=1
                data[to_remove_parent]['events'].remove(to_remove_events)
                if to_remove_events in data[to_remove_parent]['sons']:
                    data[to_remove_parent]['sons'].remove(to_remove_events)


print(remove_parent_num, remove_child_num)
            

4005 3232
1704
11
194 579


In [98]:
# Random shuffle
import random
keys = list(data.keys())
random.shuffle(keys)
new_data = {k: data[k] for k in keys}
with open("data/generated_data_fix_v5.json", 'w') as F:
    json.dump(new_data, F, indent = 4)